In [ ]:
import pandas as pd
import pickle
from scipy import stats
import pandas as pd
import seaborn as sns
import pingouin as pg
import statsmodels.formula.api as s
import numpy as np

In [ ]:
# import data from main experimental period
df = pickle.load(open("../00_data/cleaned/sitzungssummary.pkl", "rb"))
xmlsaetze = pickle.load(open("../00_data/cleaned/xmlsaetze.pkl", "rb"))
intv = df[["UserID", "interventiongroup"]]

# import data from additional experimental period
df_nov_mar = pickle.load(
    open("../00_data/competency_development/sitzungssummary_nov_mar.pkl", "rb")
)
xmlsaetze_nov_mar = pickle.load(
    open("../00_data/competency_development/xmlsaetze_nov_mar.pkl", "rb")
)
intv_nov_mar = df_nov_mar[["UserID", "interventiongroup"]]

df = pd.concat([df, df_nov_mar])
xmlsaetze = pd.concat([xmlsaetze, xmlsaetze_nov_mar])

In [ ]:
# get a list of user that had sessions in july and in late september / october
df_min = df.groupby(["UserID", "UebungsID"]).agg({"Datum": np.min})
df_max = df.groupby(["UserID", "UebungsID"]).agg({"Datum": np.max})
df_min = df_min[(df_min["Datum"] > "2022-06-21") & (df_min["Datum"] < "2022-09-21")]
df_max = df_max[(df_max["Datum"] > "2022-11-21") & (df_max["Datum"] < "2023-02-21")]
merge = pd.merge(df_min, df_max, on="UserID", how="inner")
merge = merge.reset_index()
user_both_sessions = merge[["UserID"]]
user_both_sessions = user_both_sessions.drop_duplicates()

Time period 1

In [ ]:
rasch_1 = xmlsaetze[["UserID", "SatzID", "Erfolg", "Datum"]]
rasch_1["UserID"].astype(str)
rasch_1["SatzID"].astype(str)

# only keep users in both time periods
rasch_1 = pd.merge(user_both_sessions, rasch_1, on="UserID", how="left")

# only use input from the first time
rasch_1 = rasch_1[(rasch_1["Datum"] > "2022-06-21") & (rasch_1["Datum"] < "2022-09-21")]

In [ ]:
time1 = rasch_1.groupby(["UserID", "SatzID"]).agg({"Datum": np.max})
time1 = time1.reset_index()
time1 = pd.merge(time1, rasch_1, on=["UserID", "SatzID", "Datum"])
rasch_1 = time1.pivot_table(
    values="Erfolg", index=time1.UserID, columns="SatzID", aggfunc="last"
)

In [ ]:
delete_li = []
for col in rasch_1:
    cnt = rasch_1[col].count()
    if cnt < 2:
        delete_li.append(col)

In [ ]:
rasch_1 = rasch_1.drop(
    columns=[
        9991889.0,
        9991891.0,
        9991895.0,
        9992088.0,
        9992089.0,
        9992094.0,
        9992110.0,
    ]
)

In [ ]:
delete_li = []
for index, row in rasch_1.iterrows():
    cnt = row.count()
    if cnt < 2:
        delete_li.append(index)
        rasch_1 = rasch_1.drop([index])

rasch_1

In [ ]:
rasch_1.to_csv("../00_data/competency_development/learning_data_period_1.csv")

In [ ]:
df = pd.read_csv("../00_data/competency_development/learning_data_period_1.csv")
# user id as index
df = df.set_index("UserID")
# drop rows with 0 values
df = df.dropna(how="all")
# create index with four digits
df.index = df.index.astype(str).str.pad(7, side="left", fillchar="0")
# change 1.0 to 1
df = df.replace([1.0], "1")
# change 0.0 to 0
df = df.replace([0.0], "0")
# change NaN to .
df = df.fillna(".", inplace=False)

In [ ]:
df.to_csv("../00_data/competency_development/concquest_input_data_period_1.csv")

In [ ]:
######## Manually
# open as csv in excel
## drop column row
## row to string -> delete commas
## close csv
# open csv in editor
## save as dat

Time period 2

In [ ]:
rasch_2 = xmlsaetze[["UserID", "SatzID", "Erfolg", "Datum"]]
rasch_2["UserID"].astype(str)
rasch_2["SatzID"].astype(str)

# only keeps users in both time periods
rasch_2 = pd.merge(user_both_sessions, rasch_2, on="UserID", how="left")

# only use input from the first time
rasch_2 = rasch_2[(rasch_2["Datum"] > "2022-11-21") & (rasch_2["Datum"] < "2023-02-21")]

In [ ]:
time2 = rasch_2.groupby(["UserID", "SatzID"]).agg({"Datum": np.max})
time2 = time2.reset_index()
time2 = pd.merge(time2, rasch_2, on=["UserID", "SatzID", "Datum"])

In [ ]:
rasch_2 = time2.pivot_table(
    values="Erfolg", index=time2.UserID, columns="SatzID", aggfunc="last"
)
rasch_2

In [ ]:
delete_li = []
for col in rasch_2:
    cnt = rasch_2[col].count()
    if cnt < 2:
        delete_li.append(col)

In [ ]:
delete_li = []
for index, row in rasch_2.iterrows():
    cnt = row.count()
    if cnt < 2:
        delete_li.append(index)
        rasch_2 = rasch_2.drop([index])

rasch_2

In [ ]:
rasch_2.to_csv("../00_data/competency_development/learning_data_period_2.csv")

In [ ]:
df = pd.read_csv("../00_data/competency_development/learning_data_period_2.csv")
# user id as index
df = df.set_index("UserID")
# drop rows with 0 values
df = df.dropna(how="all")
# create index with four digits
df.index = df.index.astype(str).str.pad(7, side="left", fillchar="0")
# change 1.0 to 1
df = df.replace([1.0], "1")
# change 0.0 to 0
df = df.replace([0.0], "0")
# change NaN to .
df = df.fillna(".", inplace=False)
df.to_csv("../00_data/competency_development/concquest_input_data_period_2.csv")